In [1]:
import json
import pandas as pd
import traceback

In [2]:
df=pd.read_excel('data.xlsx')
c0=df.columns
df

,序号,标注人员,标注结果
0,1,王小二,"[{""tags"":[""123"",""231""],""cc"":{""tags"":54,""data"":..."
1,2,王小二,NaN


In [3]:
df1=pd.read_excel('data1.xlsx')

df1

,序号,标注人员,标注结果
0,2,王小二,NaN
1,3,杨六,"{""tags"":[""96"",""f2931""],""username"":""楼满月"",""infos..."


In [4]:
def df_to_json(df):
    #找到json列
    jc=''
    for c in df.columns:
        a0=str(list(df[c]))
        if '{' in a0 and ':' in a0 and '[' in a0:
            jc=c
    if jc=='':
        return []
    res=[]
    for i,c in df.iterrows():
        r0={}
        for col in df.columns:
            if col!=jc:
                r0[col]=c[col]
            else:
                try:
                    if type(c[col])!=str:
                        pass
                        #r0[col]='NaN'  #[]
                    else:
                        r0[col]=json.loads(c[col])
                except:
                    print(traceback.format_exc())
        res.append(r0.copy())
    return res
r00=df_to_json(df)

In [39]:
def ex_aa(aa): 
    #aa:输入的dict, bb:公有部分,cc:上层的key,nc当前递归层数,n:限制递归层数
    flag=0
    k1=aa.keys() #初始键
    ks=[] #list对应的key
    ojs={}
    for k,v in aa.items(): #先展开dict类型,每次只展开一层，和unfolds的递归逻辑不同
        if type(v)==dict: #都没有dict也不影响，不需要加判断
            for ik,iv in v.items():
                ojs['{0}_{1}'.format(k,ik)]=iv #因为是替代掉k了，不用ojs=aa.copy()，当然也可以用del(k)的写法
        else:
            ojs[k]=v
    jsc={}
    for k,v in ojs.items(): #专门挑出list对象
        if type(v)==list:
            if v==[]:
                jsc[k]=str(v)
            elif type(v[0])==dict and k in k1:  #k in k1 用于区分这个[{},{}]到底是本层的还是ojs升上来的
                ks.append(k)
                flag+=1
            else: #多个[]镶套 
                jsc[k]=v
        else:
            jsc[k]=v
    if flag==0:
        return [jsc]
    w=[]
    for i in ojs[ks[0]]: #type(i)==dict
        w0={}  #原先递归做的{k:v,kc:[{k0:v0},{k1:v1}]} -> [{k:v,k0,v0},{k:v,k1:v1}]
        for ik,iv in i.items():
            w0['{0}_{1}'.format(ks[0],ik)]=iv
        w0.update(jsc)
        w.append(w0)
    return w

In [41]:
def unpack(jss,n=1,nc=0): #jss:[],  n:限制递归次数,nc:递归层数
    if nc>=n: #递归终止条件
        return jss
    res=[]
    for i in jss:
        try:
            res.extend(ex_aa(i))
        except:print(traceback.format_exc())
    return unpack(res,n,nc+1) 

In [42]:
res=unpack(r00,n=3)
res

[{'标注结果_infos_地点': '北京',
  '标注结果_infos_爱好': '足球',
  '标注结果_tags': ['123', '231'],
  '标注结果_cc_tags': 54,
  '标注结果_cc_data': 'cc',
  '标注结果_username': '张三',
  '序号': 1,
  '标注人员': '王小二'},
 {'标注结果_infos_地点': '天津',
  '标注结果_infos_爱好': '篮球',
  '标注结果_tags': ['123', '231'],
  '标注结果_cc_tags': 54,
  '标注结果_cc_data': 'cc',
  '标注结果_username': '张三',
  '序号': 1,
  '标注人员': '王小二'},
 {'标注结果_infos_地点': '天津',
  '标注结果_infos_爱好': '足球',
  '标注结果_tags': ['23', '2931'],
  '标注结果_username': '火星',
  '序号': 1,
  '标注人员': '王小二'},
 {'序号': 2, '标注人员': '王小二'}]

In [36]:
c0=list(df.columns)
out=pd.DataFrame(res)

for i in out.columns:
    if i not in c0:
        c0.append(i)
c1=[]
for i in c0:
    if i in out.columns:
        c1.append(i)
        
out=out.loc[:,c1]
out

,序号,标注人员,标注结果_infos_地点,标注结果_infos_爱好,标注结果_tags,标注结果_cc_tags,标注结果_cc_data,标注结果_username
0,1,王小二,北京,足球,"[123, 231]",54.0,cc,张三
1,1,王小二,天津,篮球,"[123, 231]",54.0,cc,张三
2,1,王小二,天津,足球,"[23, 2931]",NaN,NaN,火星
3,2,王小二,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
r01=df_to_json(df1)
res=unpack(r01,n=2)
c0=list(df.columns)
out=pd.DataFrame(res)

for i in out.columns:
    if i not in c0:
        c0.append(i)
c1=[]
for i in c0:
    if i in out.columns:
        c1.append(i)
        
out=out.loc[:,c1]
out

,序号,标注人员,标注结果_tags,标注结果_username,标注结果_infos
0,2,王小二,NaN,NaN,NaN
1,3,杨六,"[96, f2931]",楼满月,"[{'地点': '湖北', '爱好': '足球'}]"


In [5]:
def unfolds(js,k0='',nc=0,n=1):
    ojs={}
    if nc>n:
        return js
    for k,v in js.items():
        if type(v)==dict:
            ojs.update(unfolds(v,k))
        elif k0=='':
            ojs[k]=v
        else:
            ojs['{0}_{1}'.format(k0,k)]=v
    return ojs



def ex_aa(aa,bb,cc='',nc=0,n=1): #aa:输入的dict, bb:公有部分,cc:上层的key,nc当前递归层数,n:限制递归层数
    f=0
    if nc>=n:
        aaa=aa.copy()
        if cc!='':
            aaa={}
            for ia,iv in aa.items():
                if ia=='序号' and iv==3:
                    sadjs=1
                aaa['{0}_{1}'.format(cc,ia)]=iv
        aaa.update(bb)
        return [aaa]
    rc=bb.copy()
    ks=[]
    aa=unfolds(aa,cc,nc+1,n)
    for k,v in aa.items():
        if type(v)==dict:
            rc[k]=v #已经展开过一层
            #pass
            #默认当前层继续执行
            #aa=unfolds(aa,cc,nc+1,n)
        elif type(v)==list:
            if v==[]:
                rc[k]='[]'
            elif type(v[0])==dict:
                try:
                    kww=k.split('_')
                    if len(kww)>nc+1:
                        if cc=='':rc[k]=str(v)
                        elif rc.get(k,0)==0:rc[k]=str(v)
                        else:
                            rc['{0}_{1}'.format(cc,k)]=str(v)
                    else:
                        f+=1
                        ks.append(k) #假设有多个列表的情况
                except:print(aa,nc,traceback.format_exc())
            else:

                if cc=='':rc[k]=str(v)
                elif rc.get(k,0)==0:rc[k]=str(v)
                else:
                    rc['{0}_{1}'.format(cc,k)]=str(v)
        else:#not dict and list
            if cc=='':rc[k]=v
            elif rc.get(k,0)==0:rc[k]=v
            else:
                rc['{0}_{1}'.format(cc,k)]=v
    if f==0:
        rc.update(bb)
        return [rc]
    else:
        w=[]
        try:
            for i in aa[ks[0]]:
                w.extend(ex_aa(i,rc,ks[0],nc+1,n))
        except:print(aa,nc,traceback.format_exc())
        return w


In [6]:
res=[]
for i in r00:
    try:
        res.extend(ex_aa(i,{},'',0,3))
    except:print(traceback.format_exc())
res

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果_tags': "['123', '231']",
  'cc_tags': 54,
  'cc_data': 'cc',
  '标注结果_username': '张三',
  '标注结果_infos_地点': '北京',
  '标注结果_infos_爱好': '足球'},
 {'序号': 1,
  '标注人员': '王小二',
  '标注结果_tags': "['123', '231']",
  'cc_tags': 54,
  'cc_data': 'cc',
  '标注结果_username': '张三',
  '标注结果_infos_地点': '天津',
  '标注结果_infos_爱好': '篮球'},
 {'序号': 1,
  '标注人员': '王小二',
  '标注结果_tags': "['23', '2931']",
  '标注结果_username': '火星',
  '标注结果_infos_地点': '天津',
  '标注结果_infos_爱好': '足球'},
 {'序号': 2, '标注人员': '王小二'}]

In [7]:
c0=list(df.columns)
out=pd.DataFrame(res)

for i in out.columns:
    if i not in c0:
        c0.append(i)
c1=[]
for i in c0:
    if i in out.columns:
        c1.append(i)
        
out=out.loc[:,c1]
out

,序号,标注人员,标注结果_tags,cc_tags,cc_data,标注结果_username,标注结果_infos_地点,标注结果_infos_爱好
0,1,王小二,"['123', '231']",54.0,cc,张三,北京,足球
1,1,王小二,"['123', '231']",54.0,cc,张三,天津,篮球
2,1,王小二,"['23', '2931']",NaN,NaN,火星,天津,足球
3,2,王小二,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in out.columns:
    if i not in c0:
        c0.append(i)
c1=[]
for i in c0:
    if i in out.columns:
        c1.append(i)
        
out=out.loc[:,c1]

out.to_excel('data-out.xlsx',index=False)

## 下面是练手的

In [35]:
def unfolds(js,k0='',kvs=[]):
    ojs={}
    for k,v in js.items():
        if type(v)==dict:
            ojs1=ojs.copy()
            ojs.update(unfolds(v,k,ojs1.keys()))
        else:
            if k in kvs:
                ojs['{0}_{1}'.format(k0,k)]=v
            elif ojs.get(k,0)==0:
                ojs[k]=v
            else:#有这一个k了
                if k0=='':
                    ojs[k]=v
                else:
                    ojs['{0}_{1}'.format(k0,k)]=v
    return ojs

In [48]:
def unfolds(js,k0='',nc=0,n=1):
    ojs={}
    if nc>=n:
        return js
    for k,v in js.items():
        if type(v)==dict:
            ojs.update(unfolds(v,k))
        elif k0=='':
            ojs[k]=v
        else:
            ojs['{0}_{1}'.format(k0,k)]=v
    return ojs

In [70]:
r00

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果': [{'tags': ['123', '231'],
    'cc': {'tags': 54, 'data': 'cc'},
    'username': '张三',
    'infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}]},
   {'tags': ['23', '2931'],
    'username': '火星',
    'infos': [{'地点': '天津', '爱好': '足球'}]}]},
 {'序号': 2, '标注人员': '王小二', '标注结果': []},
 {'序号': 3,
  '标注人员': '杨六',
  '标注结果': {'tags': ['96', 'f2931'],
   'username': '楼满月',
   'infos': [{'地点': '湖北', '爱好': '足球'}]}}]

In [53]:
def ex_aa(aa,bb,cc='',nc=0,n=1): #aa:输入的dict, bb:公有部分,cc:上层的key,nc当前递归层数,n:限制递归层数
    f=0
    if nc>=n:
        return [aa]
    rc=bb.copy()
    ks=[]
    for k,v in aa.items():
        if type(v)==dict:
            #默认当前层继续执行
            aa=unfolds(aa,cc,nc+1,n)
        elif type(v)==list:
            if v==[]:
                rc[k]='[]'
            elif type(v[0])==dict:
                f+=1
                ks.append(k) #假设有多个列表的情况
            else:
                if rc.get(k,0)==0:
                    rc[k]=str(v)
                else:
                    rc['{0}_{1}'.format(cc,k)]=str(v)
        else:#not dict and list
            if rc.get(k,0)==0:
                rc[k]=v
            else:
                rc['{0}_{1}'.format(cc,k)]=v
    if f==0:
        rc.update(bb)
        return [rc]
    else:
        w=[]
        for i in aa[ks[0]]:
            w.extend(ex_aa(i,rc,ks[0],nc+1,n))
        return w

In [54]:
res=[]
for i in r00:
    res.extend(ex_aa(i,{}))
res

[{'tags': ['123', '231'],
  'cc': {'tags': 54, 'data': 'cc'},
  'username': '张三',
  'infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}]},
 {'tags': ['23', '2931'],
  'username': '火星',
  'infos': [{'地点': '天津', '爱好': '足球'}]},
 {'序号': 2, '标注人员': '王小二', '标注结果': '[]'},
 {'序号': 3, '标注人员': '杨六'}]

In [42]:
def ex_aa(aa,bb,cc=''):
    f=0
    rc=bb.copy()
    aa=unfolds(aa,cc)
    ks=[]
    for k,v in aa.items():
        if type(v)!=list:
            if rc.get(k,0)==0:
                rc[k]=v
            else:
                rc['{0}_{1}'.format(cc,k)]=v
        elif v==[]:
            pass
            #rc[k]='[]'
        else:#list
            if type(v[0])==dict:
                f+=1
                ks.append(k) #假设有多个列表的情况
            else:
                if rc.get(k,0)==0:
                    rc[k]=str(v)
                else:
                    rc['{0}_{1}'.format(cc,k)]=str(v)
    if f==0:
        rc.update(bb)
        return [rc]
    else:
        w=[]
        for i in aa[ks[0]]:
            w.extend(ex_aa(i,rc,ks[0]))
        return w

In [25]:
r00

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果': [{'tags': ['123', '231'],
    'cc': {'tags': 54, 'data': 'cc'},
    'username': '张三',
    'infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}]},
   {'tags': ['23', '2931'],
    'username': '火星',
    'infos': [{'地点': '天津', '爱好': '足球'}]}]},
 {'序号': 2, '标注人员': '王小二', '标注结果': []},
 {'序号': 3,
  '标注人员': '杨六',
  '标注结果': {'tags': ['96', 'f2931'],
   'username': '楼满月',
   'infos': [{'地点': '湖北', '爱好': '足球'}]}}]

In [55]:
pd.DataFrame(res)

,cc,infos,tags,username,序号,标注人员,标注结果
0,"{'tags': 54, 'data': 'cc'}","[{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': ...","[123, 231]",张三,NaN,NaN,NaN
1,NaN,"[{'地点': '天津', '爱好': '足球'}]","[23, 2931]",火星,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,2.0,王小二,[]
3,NaN,NaN,NaN,NaN,3.0,杨六,NaN


In [ ]:
out=pd.DataFrame(res)

In [ ]:
for i in out.columns:
    if i not in c0:
        c0.append(i)
c1=[]
for i in c0:
    if i in out.columns:
        c1.append(i)
        
out=out.loc[:,c1]

In [ ]:
out.to_excel('data-out.xlsx',index=False)

In [ ]:
df

In [56]:

def unfolds(js,k0='',nc=0,n=1):
    ojs={}
    if nc>=n:
        return js
    for k,v in js.items():
        if type(v)==dict:
            ojs.update(unfolds(v,k))
        elif k0=='':
            ojs[k]=v
        else:
            ojs['{0}_{1}'.format(k0,k)]=v
    return ojs



def ex_aa(aa,bb,cc='',nc=0,n=1): #aa:输入的dict, bb:公有部分,cc:上层的key,nc当前递归层数,n:限制递归层数
    f=0
    if nc>=n:
        aaa=aa.copy()
        if cc!='':
            aaa={}
            for ia,iv in aa.items():
                aaa['{0}_{1}'.format(cc,ia)]=iv
        aaa.update(bb)
        return [aaa]
    rc=bb.copy()
    ks=[]
    for k,v in aa.items():
        if type(v)==dict:
            #默认当前层继续执行
            aa=unfolds(aa,cc,nc+1,n)
        elif type(v)==list:
            if v==[]:
                rc[k]='[]'
            elif type(v[0])==dict:
                f+=1
                ks.append(k) #假设有多个列表的情况
            else:
                if rc.get(k,0)==0:
                    rc[k]=str(v)
                else:
                    rc['{0}_{1}'.format(cc,k)]=str(v)
        else:#not dict and list
            if rc.get(k,0)==0:
                rc[k]=v
            else:
                rc['{0}_{1}'.format(cc,k)]=v
    if f==0:
        rc.update(bb)
        return [rc]
    else:
        w=[]
        for i in aa[ks[0]]:
            w.extend(ex_aa(i,rc,ks[0],nc+1,n))
        return w

res=[]
for i in r00:
    res.extend(ex_aa(i,{}))
print(res)




[{'标注结果_tags': ['123', '231'], '标注结果_cc': {'tags': 54, 'data': 'cc'}, '标注结果_username': '张三', '标注结果_infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}], '序号': 1, '标注人员': '王小二'}, {'标注结果_tags': ['23', '2931'], '标注结果_username': '火星', '标注结果_infos': [{'地点': '天津', '爱好': '足球'}], '序号': 1, '标注人员': '王小二'}, {'序号': 2, '标注人员': '王小二', '标注结果': '[]'}, {'序号': 3, '标注人员': '杨六'}]


In [57]:
pd.DataFrame(res)

,序号,标注人员,标注结果,标注结果_cc,标注结果_infos,标注结果_tags,标注结果_username
0,1,王小二,NaN,"{'tags': 54, 'data': 'cc'}","[{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': ...","[123, 231]",张三
1,1,王小二,NaN,NaN,"[{'地点': '天津', '爱好': '足球'}]","[23, 2931]",火星
2,2,王小二,[],NaN,NaN,NaN,NaN
3,3,杨六,NaN,NaN,NaN,NaN,NaN


In [20]:
r00

[{'序号': 1,
  '标注人员': '王小二',
  '标注结果': [{'tags': ['123', '231'],
    'cc': {'tags': 54, 'data': 'cc'},
    'username': '张三',
    'infos': [{'地点': '北京', '爱好': '足球'}, {'地点': '天津', '爱好': '篮球'}]},
   {'tags': ['23', '2931'],
    'username': '火星',
    'infos': [{'地点': '天津', '爱好': '足球'}]}]},
 {'序号': 2, '标注人员': '王小二', '标注结果': []},
 {'序号': 3,
  '标注人员': '杨六',
  '标注结果': {'tags': ['96', 'f2931'],
   'username': '楼满月',
   'infos': [{'地点': '湖北', '爱好': '足球'}]}}]

In [41]:
aa={'a':[{'aa':1,'bb':[{'aaa':31,'bbb':32},{'aaa':41,'bbb':42}]},{'aa':11}],'b':2}
ex_aa(aa,{})

[{'b': 2, 'aa': 1, 'aaa': 31, 'bbb': 32},
 {'b': 2, 'aa': 1, 'aaa': 41, 'bbb': 42},
 {'b': 2, 'aa': 11}]

In [34]:
aa={'a':[{'aa':1,'bb':[{'aaa':31,'bbb':32},{'aaa':41,'bbb':42}]},{'aa':11}],'b':2}
'a' in aa.keys()

True